In [1]:
import os

import copy
"""
Assignment statements in Python do not copy objects!
they create bindings between a target and an object.
a copy is sometimes needed so one can change one copy without changing the other!
"""
from pathlib import Path

import scipy

import krotov
import krotov as kr

import numpy as np
import qutip
import matplotlib.pyplot as plt
#import colorsys
#from mpl_toolkits.mplot3d import Axes3D

import qdyn.model
import qdyn.pulse
# QDYN-pylib is a Python package for interacting with the Fortran QDYN library and tools
#!make clean

from multiprocessing import Pool

from qutip import tensor,qeye,ket
#from qutip.qip.circuit import t_gate
from qutip.qip.operations import berkeley,swap

import subprocess as subp

In [2]:
#%matplotlib widget

In [3]:
#final time
τ = 1.8


#t_rise
tr = τ/10

#delta_t
Δt = 0.0018

#lambda
λ = 0.07

# Number of time points
n = int(τ/Δt + 1)


In [4]:
#Target-Gate

B = berkeley(2)

E = qeye(2)

S = swap(2)

U = tensor(S,E)*tensor(E,B)*tensor(S,E)


Target = U

#Target = tensor(B,E)
Target = np.array(Target)

target_gate = qutip.Qobj(Target)



# Initialize List
L = [τ,tr,Δt,λ,target_gate]



In [5]:
# lambda t - "anonymous" function of t

def get_Ham(ν1=5.0, ν2 = 5.3, ν3 = 5.5, T = τ, t_rise = tr): 

    μ1 = -15.377
    μ2 = 4.296
    μ3 = 5.353
    μ4 = 5.729
    μ5 = 16.26
    μ6 = -4.09
    μ7 = -5.804
    μ8 = -6.360


    N = 1.131234587427
    
    u = lambda t, args: 0.05*N*(np.pi)*(8/T)*kr.shapes.flattop(t, t_start=0.0, t_stop=T, t_rise = t_rise, t_fall = t_rise, func='sinsq')*(np.cos(μ2*2*np.pi*t) + np.sin(μ3*2*np.pi*t) - np.sin(μ4*2*np.pi*t)) + N*(np.pi/2)*(8/T)*kr.shapes.flattop(t, t_start=0, t_stop=T/8, t_rise = t_rise, t_fall = t_rise, func='blackman')*np.sin(μ1*2*np.pi*t) + N*(np.pi/2)*(8/T)*kr.shapes.flattop(t, t_start=T/8, t_stop=2*T/8, t_rise = t_rise, t_fall = t_rise, func='blackman')*np.cos(μ2*2*np.pi*t)**2 -  N*(np.pi/2)*(8/T)*kr.shapes.flattop(t, t_start=2*T/8, t_stop=3*T/8, t_rise = t_rise, t_fall = t_rise, func='blackman')*np.sin(μ1*2*np.pi*t)**3 + N*(np.pi/2)*(8/T)*kr.shapes.flattop(t, t_start=3*T/8, t_stop=4*T/8, t_rise = t_rise, t_fall = t_rise, func='sinsq')*np.cos(μ3*2*np.pi*t)**4 - N*(np.pi/2)*(8/T)*kr.shapes.flattop(t, t_start=4*T/8, t_stop=5*T/8, t_rise = t_rise, t_fall = t_rise, func='blackman')*np.sin(μ7*2*np.pi*t)**5 +  N*(np.pi/2)*(8/T)*kr.shapes.flattop(t, t_start=5*T/8, t_stop=6*T/8, t_rise = t_rise, t_fall = t_rise, func='blackman')*np.cos(μ4*2*np.pi*t)**4  +  N*(np.pi/2)*(8/T)*kr.shapes.flattop(t, t_start=6*T/8, t_stop=7*T/8, t_rise = t_rise, t_fall = t_rise, func='sinsq')*np.sin(μ8*2*np.pi*t)**3 +  N*(np.pi/2)*(8/T)*kr.shapes.flattop(t, t_start=7*T/8, t_stop=T, t_rise = t_rise, t_fall = t_rise, func='blackman')*np.cos(μ5*2*np.pi*t)**2



    
    #N = 1.131234587427
    # coupling
    u_ent12 = lambda t, args: N*krotov.shapes.flattop(t, t_start=0.0, t_stop=T/4, t_rise = t_rise, t_fall = t_rise, func='blackman')*(np.pi/2*(1/(T/4))*(np.cos(ν1*t*2*np.pi) + np.cos(ν2*t*2*np.pi))) + N*krotov.shapes.flattop(t, t_start=T/3, t_stop=T, t_rise = t_rise, t_fall = t_rise, func='blackman')*(np.pi/2*(1/(T/3))*(np.cos(ν1*t*2*np.pi) - np.sin(ν2*t*2*np.pi)))
    v_ent12 = lambda t, args: N*krotov.shapes.flattop(t, t_start=0.0, t_stop=T/4, t_rise = t_rise, t_fall = t_rise, func='blackman')*(np.pi/2*(1/(T/4))*(np.sin(ν1*t*2*np.pi) - np.cos(ν2*t*2*np.pi))) + N*krotov.shapes.flattop(t, t_start=T/3, t_stop=T, t_rise = t_rise, t_fall = t_rise, func='blackman')*(np.pi/2*(1/(T/3))*(np.cos(ν1*t*2*np.pi) + np.sin(ν2*t*2*np.pi)))
    

   #u_tgt = lambda t, args: N*krotov.shapes.flattop(t, t_start=0.0, t_stop=T/4, t_rise = t_rise, t_fall = t_rise, func='blackman')*(np.pi/2*(1/(T/4))*(np.cos(ν1*t*2*np.pi) + np.cos(ν2*t*2*np.pi)) + N*krotov.shapes.flattop(t, t_start=T/2, t_stop=T, t_rise = t_rise, t_fall = t_rise, func='blackman')*(np.pi/2*(1/(T/2))*(np.cos(ν1*t*2*np.pi) - np.sin(ν2*t*2*np.pi)) 
    u_id1 = lambda t, args: N*krotov.shapes.flattop(t, t_start=0.0, t_stop=T, t_rise = t_rise, t_fall = t_rise, func='blackman')*4*np.pi/T*(np.cos(ν1*t*2*np.pi))
    u_id2 = lambda t, args: N*krotov.shapes.flattop(t, t_start=0.0, t_stop=T, t_rise = t_rise, t_fall = t_rise, func='blackman')*4*np.pi/T*(np.cos(ν2*t*2*np.pi)) 
    u_id3 = lambda t, args: N*krotov.shapes.flattop(t, t_start=0.0, t_stop=T, t_rise = t_rise, t_fall = t_rise, func='blackman')*4*np.pi/T*(np.cos(ν3*t*2*np.pi))

    u_3 = lambda t, args: N*krotov.shapes.flattop(t, t_start=0.0, t_stop=T/4, t_rise = t_rise, t_fall = t_rise, func='blackman')*(np.pi/2*(1/(T/4))*(np.cos(ν3*t*2*np.pi))) + N*krotov.shapes.flattop(t, t_start=T/3, t_stop=T, t_rise = t_rise, t_fall = t_rise, func='blackman')*(np.pi/2*(1/(T/3)))*(np.sin(ν3*t*2*np.pi))
    u_2 = lambda t, args: N*krotov.shapes.flattop(t, t_start=0.0, t_stop=T/4, t_rise = t_rise, t_fall = t_rise, func='blackman')*(np.pi/2*(1/(T/4))*(np.cos(ν2*t*2*np.pi))) + N*krotov.shapes.flattop(t, t_start=T/3, t_stop=T, t_rise = t_rise, t_fall = t_rise, func='blackman')*(np.pi/2*(1/(T/3)))*(np.sin(ν2*t*2*np.pi))
    u_1 = lambda t, args: N*krotov.shapes.flattop(t, t_start=0.0, t_stop=T/4, t_rise = t_rise, t_fall = t_rise, func='blackman')*(np.pi/2*(1/(T/4))*(np.sin(ν1*t*2*np.pi))) + N*krotov.shapes.flattop(t, t_start=T/3, t_stop=T, t_rise = t_rise, t_fall = t_rise, func='blackman')*(np.pi/2*(1/(T/3)))*(np.cos(ν1*t*2*np.pi))
                                                                                                                                                                                                                                                                                  
    
    Z = qutip.operators.sigmaz()
    X = qutip.operators.sigmax()
    E = qeye(2) 
    
    X1X2 = tensor(X,X,E)
    Z1Z2 = tensor(Z,Z,E)
    
    X2X3 = tensor(E,X,X)
    
   
    
 
    k1 = 0.112358
    k2 = 0.222222
    k3 = 0.314159
    
    
    drift = -1/2 * ( ν1*tensor(Z,E,E) + ν2*tensor(E,Z,E) + ν3*tensor(E,E,Z) ) 
    coupl = k1*X1X2 + k2*Z1Z2 + k3*X2X3 
    
    H0 = drift + coupl
    H0 = np.array(H0)  # important for dimensionality. Code recognizes the 8 x 
    
    Ham_0 = qutip.Qobj(H0) 

    H1 = tensor(X,E,E)
    H1 = np.array(H1)
    Ham_1 = qutip.Qobj(H1)
    

    H2 = tensor(E,X,E)
    H2 = np.array(H2)
    Ham_2 = qutip.Qobj(H2)
    
    H3 = tensor(E,E,X)
    H3 = np.array(H3)
    Ham_3 = qutip.Qobj(H3)

    
    return [Ham_0, [Ham_1, np.vectorize(u)],[Ham_2, np.vectorize(u_id2)],[Ham_3, np.vectorize(u)]]

In [6]:
def qdyn_model(
    Ham,
    T, # final_time
    nt, # number of time steps
    basis, # logical basis
    gate, # target gate
    runfolder,
    prop_method="exact",
    J_T_conv=1e-4,
    Δ_J_T_conv = 1e-7,
    iter_stop=100000,
    **pulse_oct_kwargs, # kwargs = keyword arguments
):
    """ Create qdyn-model

    """
    runfolder = Path(runfolder) # path to runfolder
    ! mkdir $runfolder # bash command 
    tgrid = qdyn.pulse.pulse_tgrid(T=T, nt=nt, t0=0)
      # For technical reasons, QDYN stores the pulses on the intervals of the time grid.
    # That means, the time grid for the pulse needs to be shifted
    #by dt/2 with respect to the time grid used for the propagation
    
    model = qdyn.model.LevelModel()
    # 
    for H in Ham:
        if isinstance(H, list):
            model.add_ham(
                H[0],
                pulse=qdyn.pulse.Pulse(
                    tgrid,
                    amplitude=H[1](tgrid, None),
                    time_unit="ns",
                    ampl_unit="GHz",
                    config_attribs={
                        **pulse_oct_kwargs
                    }, # Additional config data, for when generating a QDYN config
                    # file section describing the pulse (e.g. {'oct_shape': 'flattop', 't_rise': '10_ns'})
                ),
                op_unit="iu",
            )
        else:
            model.add_ham(H, op_unit="GHz")
            
    qdyn.io.write_cmplx_array(
        qutip.Qobj(gate).full().flatten(order="F"), str(runfolder/"gate.dat")
    )
    model.set_propagation(
        T, nt, time_unit="ns", prop_method=prop_method
    )
    model.set_oct("krotovpk", J_T_conv, 5000, iter_stop=iter_stop, continue_=False, delta_J_T_conv=Δ_J_T_conv)
    
    # Observables
    Z = qutip.operators.sigmaz()
    E = qeye(2)
    
    Z1 = qutip.Qobj(np.array(tensor(Z,E,E)))
    Z2 = qutip.Qobj(np.array(tensor(E,Z,E)))
    Z3 = qutip.Qobj(np.array(tensor(E,E,Z)))
    



    model.add_observable(Z1, "observable.dat", exp_unit="GHz", time_unit="ns", col_label = "1" )
    model.add_observable(Z2, "observable.dat", exp_unit="GHz", time_unit="ns", col_label = "2" )
    model.add_observable(Z3, "observable.dat", exp_unit="GHz", time_unit="ns", col_label = "3" )
    
    
    
    
    for i, b in enumerate(basis):
        model.add_state(b, f"basis{i}")
        

    Path(runfolder).mkdir(exist_ok=True, parents=True)
    model.write_to_runfolder(str(runfolder))

In [ ]:

def create_runfolder(L):
    
    [τ,tr,Δt,λ,target_gate] = L

    pulse_oct_kwargs = dict(oct_lambda_a=λ, oct_shape = "flattop",t_rise=qdyn.units.UnitFloat(τ/10, "ns"), t_fall=qdyn.units.UnitFloat(τ/10, "ns") )
    
    rf_p = "rf"+str(τ)
    
    qdyn_model(get_Ham(T = τ, t_rise = τ/10), τ,  int(τ/Δt + 1),[qutip.basis(8,i) for i in range(0,8)], target_gate, rf_p  , **pulse_oct_kwargs)

   

def do_final_times(a, b, c):
    """
    a = final time
    b = decreasing parameter
    c = how often shall b be reduced from a
    """
    return np.arange(a, 0, -b)[:c]

def do_optimization_input(L,b,c):
 
    a = L[0]
    P = [[m] + L[1:5] for m in do_final_times(a, b, c)]
    
    return P

def make_multiple_runfolders(L,b,c):
    input_list = do_optimization_input(L,b,c)
    
    for i,L_i in enumerate(input_list):
        rf_i = create_runfolder(L_i)
    print('Ready')

#make_multiple_runfolders(L,1,5)
make_multiple_runfolders(L,0.2,6)


#! mkdir All_runfolders
#! mkdir job_msgs
    